<a href="https://colab.research.google.com/github/Tower5954/TensorFlow/blob/main/04_Transfer_learning_in_tensorflow_part_1_feature_extraction_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning with TensorFlow part 1: Feature extraction.

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problem.

There are 2 main benefits:

1. Can leverage an existing neural network architecture proven to work on problems similar to our own.
2. Can leverage a working neural network architecture which has already learned patterns on similar data to our own, then we can adapt those patterns to our own data.

In [1]:
# Are we using a GPU?
!nvidia-smi

Sat Dec 11 03:36:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading and becoming one with the data.



In [2]:
# Get data (10% of 10 food classes from Food101)

import zipfile

# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()


--2021-12-11 03:36:24--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  51.2MB/s    in 3.1s    

2021-12-11 03:36:28 (51.2 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are: {len(dirnames)} directories and {len(filenames)} images in '{dirpath}' ")

There are: 2 directories and 0 images in '10_food_classes_10_percent' 
There are: 10 directories and 0 images in '10_food_classes_10_percent/test' 
There are: 0 directories and 250 images in '10_food_classes_10_percent/test/sushi' 
There are: 0 directories and 250 images in '10_food_classes_10_percent/test/pizza' 
There are: 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger' 
There are: 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings' 
There are: 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream' 
There are: 0 directories and 250 images in '10_food_classes_10_percent/test/steak' 
There are: 0 directories and 250 images in '10_food_classes_10_percent/test/ramen' 
There are: 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice' 
There are: 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry' 
There are: 0 directories and 250 images in '10_food_classes_10_perc

## Create data loaders (preparing the data)

We will use the `ImageDataGenerator` class to load in our images in batches 

In [4]:
# Set up data inputs

from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = (32)

train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical")


print("Testing images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")

Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


## Setting up callbacks (things to run while the model trains)

Callbacks are extra functionality you can add to a model to be performed during or after training. Some of the more popular callbacks are:
* Tracking experiments with the TensorBoard callback.
* Model checkpoint with the ModelCheckpoint callback.
* Stopping a model from training (before it trains too long and overfits) with the EarlyStopping callback.





In [5]:
# Create TensorBoard callback (functionised to allow for creating a new one for each model)

In [6]:
import datetime
import tensorflow as tf


def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" +experiment_name + "/" + datetime.datetime.now().strftime('%d%m%Y-%H%M')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir)
  print(f"Saving Tensorboard log files to: {log_dir}")
  return tensorboard_callback

🔑 **Note**: 🔑 You can customise the directory where your TensorBoard logs(model training metrics)gets saved to wherever you like. The `log_dir` parameter created above is just one option. 